# USAD

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import logging
import os
import sys
import warnings
import torch.utils.data as data_utils
from sklearn import preprocessing
from tqdm import tqdm
import os
import time

# Ensure USAD and device are imported

sys.path.append(os.path.abspath("../.."))
from spectrum.models import USAD
from spectrum.utils import device
from spectrum.utils.random import set_random_state
from spectrum.models import USAD
from spectrum.utils import device
logging.basicConfig(level=logging.INFO)

warnings.filterwarnings("ignore")

sns.set_theme(style="whitegrid")
plt.rcParams.update(
    {
        "axes.edgecolor": "0.3",
        "axes.linewidth": 0.8,
        "font.size": 12,
        "axes.titlesize": 14,
        "axes.labelsize": 12,
        "axes.titleweight": "bold",
        "legend.fontsize": 10,
        "figure.dpi": 120,
        "legend.frameon": False,
    }
)

set_random_state()

In [4]:
BATCH_SIZE = 1024
N_EPOCHS = 30
HIDDEN_SIZE = 100
WINDOW_SIZE = 12

results_dir = "../../results/models/usad"
os.makedirs(results_dir, exist_ok=True)
selected_ids = [1, 2, 3]

def find_best_threshold(scores, true_labels, thresholds=None):
    if thresholds is None:
        # Handle case where scores are all same (e.g. 0)
        if np.min(scores) == np.max(scores):
            thresholds = [scores[0]]
        else:
            thresholds = [np.percentile(scores, p) for p in range(0, 90, 5)]
            thresholds.extend([np.percentile(scores, p) for p in range(90, 100, 1)])
            thresholds.extend([np.percentile(scores, p) for p in [99.1, 99.3, 99.5, 99.7, 99.9, 99.95, 99.99]])
            
    thresholds = sorted(list(set(thresholds)), reverse=True)

    best_f1 = -1
    best_threshold = thresholds[0] if len(thresholds) > 0 else 0.0
    
    best_metrics = {
        'threshold': best_threshold,
        'accuracy': 0.0,
        'precision': 0.0,
        'recall': 0.0,
        'f1': 0.0,
        'fnr': 0.0,
        'fpr': 0.0,
        'tp': 0,
        'fp': 0,
        'tn': 0,
        'fn': 0
    }

    for threshold in thresholds:
        pred_labels = (scores > threshold).astype(int)
        TP = ((true_labels == 1) & (pred_labels == 1)).sum()
        FP = ((true_labels == 0) & (pred_labels == 1)).sum()
        TN = ((true_labels == 0) & (pred_labels == 0)).sum()
        FN = ((true_labels == 1) & (pred_labels == 0)).sum()

        accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        fnr = FN / (FN + TP) if (FN + TP) > 0 else 0
        fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
        
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_metrics = {
                'threshold': threshold,
                'accuracy': accuracy,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'fnr': fnr,
                'fpr': fpr,
                'tp': TP,
                'fp': FP,
                'tn': TN,
                'fn': FN
            }
            
    return best_threshold, best_metrics

def process_single_id(data_id):
    print(f"processing: {data_id}")
    
    # Read data
    train_df_raw = pd.read_csv(f"../../datasets/Tencent/train/{data_id}.csv")
    test_df_raw = pd.read_csv(f"../../datasets/Tencent/test/{data_id}.csv")
    
    # Prepare Train Data
    train_vals = train_df_raw.drop(["timestamp", "label"], axis=1, errors='ignore')
    train_vals = train_vals.astype(float)
    
    # Prepare Test Data
    test_vals = test_df_raw.drop(["timestamp", "label"], axis=1, errors='ignore')
    test_vals = test_vals.astype(float)
    
    # Normalization
    min_max_scaler = preprocessing.MinMaxScaler()
    x_train = min_max_scaler.fit_transform(train_vals.values)
    x_test = min_max_scaler.transform(test_vals.values)
    
    # Create Windows
    def make_windows(data_arr):
        # data_arr: (N, F)
        n = data_arr.shape[0]
        if n <= WINDOW_SIZE:
            return np.empty((0, WINDOW_SIZE, data_arr.shape[1]))
        indexer = np.arange(WINDOW_SIZE)[None, :] + np.arange(n - WINDOW_SIZE + 1)[:, None]
        return data_arr[indexer]

    train_windows = make_windows(x_train)
    test_windows = make_windows(x_test)
    
    # Flatten windows for USAD
    w_size = WINDOW_SIZE * x_train.shape[1]
    z_size = WINDOW_SIZE * HIDDEN_SIZE
    
    train_windows_flat = train_windows.reshape(-1, w_size)
    test_windows_flat = test_windows.reshape(-1, w_size)
    
    # Split Train/Val
    split_idx = int(0.8 * len(train_windows_flat))
    train_loader = torch.utils.data.DataLoader(
        data_utils.TensorDataset(
            torch.from_numpy(train_windows_flat[:split_idx]).float()
        ),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    val_loader = torch.utils.data.DataLoader(
        data_utils.TensorDataset(
            torch.from_numpy(train_windows_flat[split_idx:]).float()
        ),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    test_loader = torch.utils.data.DataLoader(
        data_utils.TensorDataset(
            torch.from_numpy(test_windows_flat).float()
        ),
        batch_size=BATCH_SIZE,
        shuffle=False
    )
    
    # Train
    model = USAD(N_EPOCHS, w_size, z_size).to(device())
    
    print("  training model...")
    start_time = time.time()
    history = model.fit(train_loader, val_loader)
    training_time = time.time() - start_time
    
    print("  anomaly detection...")
    start_time = time.time()
    results_list = model.predict(test_loader)
    if len(results_list) > 0:
        scores = torch.cat(results_list).cpu().numpy()
    else:
        scores = np.array([])
    scoring_time = time.time() - start_time
    
    print(f"  Scores stats: min={scores.min():.6f}, max={scores.max():.6f}, mean={scores.mean():.6f}")
    
    # Window-based Labeling Logic
    test_true_labels = test_df_raw["label"].to_numpy()
    
    # Create windows of labels using the same make_windows function
    # We reshape labels to (N, 1) then flatten the result to (num_windows, WINDOW_SIZE)
    label_windows = make_windows(test_true_labels.reshape(-1, 1)).reshape(-1, WINDOW_SIZE)
    
    # If ANY point in window is anomaly (1), then window label is 1
    y_test = (label_windows.sum(axis=1) > 0).astype(int)
    
    # Align lengths
    min_len = min(len(scores), len(y_test))
    scores = scores[:min_len]
    y_test = y_test[:min_len]

    print("  finding best threshold...")
    best_threshold, best_metrics = find_best_threshold(scores, y_test)
    
    # Save Results (Complete Data)
    full_df = pd.concat([train_df_raw, test_df_raw])
    
    if "value" in full_df.columns:
        complete_values = full_df["value"].to_numpy()
    else:
        complete_values = full_df.iloc[:, 1].to_numpy() # Use first feature column
        
    if "label" in full_df.columns:
        complete_labels = full_df["label"].to_numpy()
    else:
        complete_labels = np.zeros(len(complete_values))
        
    if "timestamp" in full_df.columns:
        complete_timestamps = full_df["timestamp"].to_numpy()
    else:
        complete_timestamps = range(len(complete_values))
        
    complete_predictions = np.zeros(len(complete_values))
    complete_anomaly_scores = np.zeros(len(complete_values))
    
    # Align predictions
    # Test scores correspond to windows. 
    # We align the score to the END of the window (point-based alignment for visualization)
    # Start index in full_df: len(train) + WINDOW_SIZE - 1
    
    pred_start_idx = len(train_df_raw) + WINDOW_SIZE - 1
    
    end_idx = pred_start_idx + len(scores)
    if end_idx > len(complete_values):
        end_idx = len(complete_values)
        scores = scores[:end_idx - pred_start_idx]
        
    complete_predictions[pred_start_idx:end_idx] = (scores > best_threshold).astype(int)
    complete_anomaly_scores[pred_start_idx:end_idx] = scores
    
    result_df = pd.DataFrame({
        'timestamp': complete_timestamps,
        'value': complete_values,
        'label': complete_labels,
        'predicted': complete_predictions,
        'anomaly_score': complete_anomaly_scores
    })
    
    output_file = os.path.join(results_dir, f"{data_id}.csv")
    result_df.to_csv(output_file, index=False)
    print(f"  results saved to: {output_file}")
    
    return {
        'id': data_id,
        'training_time': training_time,
        'testing_time': scoring_time,
        'total_time': training_time + scoring_time,
        'train_samples': len(train_df_raw),
        'test_samples': len(test_df_raw),
        'best_threshold': best_threshold,
        **best_metrics
    }

# Main Loop
all_results = []
print(f"processing {len(selected_ids)} datasets...")

for data_id in tqdm(selected_ids, desc="processing"):
    try:
        result = process_single_id(data_id)
        all_results.append(result)
        
        print(f"  ID {data_id} completed:")
        print(f"    best_threshold: {result['best_threshold']:.4f}")
        print(f"    f1: {result['f1']:.4f}")
        print(f"    precision: {result['precision']:.4f}")
        print(f"    recall: {result['recall']:.4f}")
        print(f"    accuracy: {result['accuracy']:.4f}")
        
    except Exception as e:
        print(f"  processing {data_id} failed: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

if all_results:
    summary_df = pd.DataFrame(all_results)
    summary_file = os.path.join(results_dir, "usad.csv")
    summary_df.to_csv(summary_file, index=False)
    print(f"summary results saved to: {summary_file}")
    
    print("\n" + "=" * 80)
    print("USAD anomaly detection results")
    print("=" * 80)
    print(f"processed {len(all_results)} datasets")
    print(f"average F1: {summary_df['f1'].mean():.4f} ± {summary_df['f1'].std():.4f}")
    print(f"average precision: {summary_df['precision'].mean():.4f} ± {summary_df['precision'].std():.4f}")
    print(f"average recall: {summary_df['recall'].mean():.4f} ± {summary_df['recall'].std():.4f}")
    print(f"average accuracy: {summary_df['accuracy'].mean():.4f} ± {summary_df['accuracy'].std():.4f}")
    print(f"average training time: {summary_df['training_time'].mean():.2f}s")
    print(f"average scoring time: {summary_df['testing_time'].mean():.2f}s")
    print("=" * 80)
    
    print("details:")
    display_cols = ['id', 'f1', 'precision', 'recall', 'accuracy', 'best_threshold', 'tp', 'fp', 'tn', 'fn']
    print(summary_df[display_cols].round(4))
else:
    print("no results")

processing 3 datasets...


processing:   0%|          | 0/3 [00:00<?, ?it/s]

processing: 1
  training model...
Epoch [0], val_loss1: 0.2475, val_loss2: 0.2502
Epoch [1], val_loss1: 0.2444, val_loss2: 0.0000
Epoch [2], val_loss1: 0.2406, val_loss2: -0.0810
Epoch [3], val_loss1: 0.2365, val_loss2: -0.1197
Epoch [4], val_loss1: 0.2321, val_loss2: -0.1414
Epoch [5], val_loss1: 0.2273, val_loss2: -0.1545
Epoch [6], val_loss1: 0.2222, val_loss2: -0.1625
Epoch [7], val_loss1: 0.2167, val_loss2: -0.1671
Epoch [8], val_loss1: 0.2105, val_loss2: -0.1691
Epoch [9], val_loss1: 0.2038, val_loss2: -0.1692
Epoch [10], val_loss1: 0.1963, val_loss2: -0.1673
Epoch [11], val_loss1: 0.1876, val_loss2: -0.1633
Epoch [12], val_loss1: 0.1783, val_loss2: -0.1580
Epoch [13], val_loss1: 0.1684, val_loss2: -0.1515
Epoch [14], val_loss1: 0.1581, val_loss2: -0.1439
Epoch [15], val_loss1: 0.1468, val_loss2: -0.1350
Epoch [16], val_loss1: 0.1348, val_loss2: -0.1250
Epoch [17], val_loss1: 0.1226, val_loss2: -0.1144
Epoch [18], val_loss1: 0.1100, val_loss2: -0.1032
Epoch [19], val_loss1: 0.097

processing:  33%|███▎      | 1/3 [00:05<00:10,  5.27s/it]

Epoch [29], val_loss1: 0.0162, val_loss2: -0.0156
  anomaly detection...
  Scores stats: min=0.010234, max=907494.750000, mean=2772.411377
  finding best threshold...
  results saved to: ../../results/models/usad/1.csv
  ID 1 completed:
    best_threshold: 0.0102
    f1: 1.0000
    precision: 1.0000
    recall: 1.0000
    accuracy: 1.0000
processing: 2
  training model...
Epoch [0], val_loss1: 0.2372, val_loss2: 0.2424
Epoch [1], val_loss1: 0.2314, val_loss2: 0.0001
Epoch [2], val_loss1: 0.2240, val_loss2: -0.0762
Epoch [3], val_loss1: 0.2152, val_loss2: -0.1100
Epoch [4], val_loss1: 0.2051, val_loss2: -0.1260
Epoch [5], val_loss1: 0.1938, val_loss2: -0.1325
Epoch [6], val_loss1: 0.1812, val_loss2: -0.1328
Epoch [7], val_loss1: 0.1664, val_loss2: -0.1281
Epoch [8], val_loss1: 0.1498, val_loss2: -0.1194
Epoch [9], val_loss1: 0.1324, val_loss2: -0.1084
Epoch [10], val_loss1: 0.1151, val_loss2: -0.0962
Epoch [11], val_loss1: 0.0986, val_loss2: -0.0838
Epoch [12], val_loss1: 0.0832, val_lo

processing:  67%|██████▋   | 2/3 [00:11<00:05,  5.71s/it]

Epoch [29], val_loss1: 0.0021, val_loss2: -0.0019
  anomaly detection...
  Scores stats: min=0.002832, max=0.908030, mean=0.005790
  finding best threshold...
  results saved to: ../../results/models/usad/2.csv
  ID 2 completed:
    best_threshold: 0.0028
    f1: 1.0000
    precision: 1.0000
    recall: 1.0000
    accuracy: 1.0000
processing: 3
  training model...
Epoch [0], val_loss1: 0.2419, val_loss2: 0.2486
Epoch [1], val_loss1: 0.2404, val_loss2: 0.0000
Epoch [2], val_loss1: 0.2363, val_loss2: -0.0802
Epoch [3], val_loss1: 0.2310, val_loss2: -0.1178
Epoch [4], val_loss1: 0.2243, val_loss2: -0.1375
Epoch [5], val_loss1: 0.2158, val_loss2: -0.1473
Epoch [6], val_loss1: 0.2057, val_loss2: -0.1507
Epoch [7], val_loss1: 0.1940, val_loss2: -0.1495
Epoch [8], val_loss1: 0.1806, val_loss2: -0.1445
Epoch [9], val_loss1: 0.1659, val_loss2: -0.1367
Epoch [10], val_loss1: 0.1501, val_loss2: -0.1266
Epoch [11], val_loss1: 0.1335, val_loss2: -0.1148
Epoch [12], val_loss1: 0.1165, val_loss2: -0.

processing: 100%|██████████| 3/3 [00:15<00:00,  5.11s/it]

Epoch [28], val_loss1: 0.0039, val_loss2: -0.0036
Epoch [29], val_loss1: 0.0035, val_loss2: -0.0033
  anomaly detection...
  Scores stats: min=0.005141, max=0.908112, mean=0.008081
  finding best threshold...
  results saved to: ../../results/models/usad/3.csv
  ID 3 completed:
    best_threshold: 0.9078
    f1: 0.0000
    precision: 0.0000
    recall: 0.0000
    accuracy: 0.9998
summary results saved to: ../../results/models/usad/usad.csv

USAD anomaly detection results
processed 3 datasets
average F1: 0.6667 ± 0.5774
average precision: 0.6667 ± 0.5774
average recall: 0.6667 ± 0.5774
average accuracy: 0.9999 ± 0.0001
average training time: 5.03s
average scoring time: 0.03s
details:
   id   f1  precision  recall  accuracy  best_threshold  tp  fp    tn  fn
0   1  1.0        1.0     1.0    1.0000          0.0102  66   0  4243   0
1   2  1.0        1.0     1.0    1.0000          0.0028  66   0  4243   0
2   3  0.0        0.0     0.0    0.9998          0.9078   0   1  4308   0


In [5]:
def process_synthetic_data_usad():
    print("\n" + "="*40)
    print("Processing Synthetic Dataset (USAD)...")
    print("="*40)
    
    # Paths
    train_path = "../../datasets/synthetic/train/train.csv"
    test_path = "../../datasets/synthetic/test/test.csv"
    
    if not os.path.exists(train_path):
        print(f"Error: {train_path} not found. Please run preprocess/synthetic.ipynb first.")
        return

    # Read data
    train_df_raw = pd.read_csv(train_path)
    test_df_raw = pd.read_csv(test_path)
    
    print(f"  Train shape: {train_df_raw.shape}")
    print(f"  Test shape: {test_df_raw.shape}")
    
    # Prepare Numeric Data
    # Drop timestamp and label
    cols_to_drop = ["timestamp", "label"]
    train_vals = train_df_raw.drop([c for c in cols_to_drop if c in train_df_raw.columns], axis=1)
    test_vals = test_df_raw.drop([c for c in cols_to_drop if c in test_df_raw.columns], axis=1)
    
    train_vals = train_vals.astype(float)
    test_vals = test_vals.astype(float)
    
    # Normalization
    min_max_scaler = preprocessing.MinMaxScaler()
    x_train = min_max_scaler.fit_transform(train_vals.values)
    x_test = min_max_scaler.transform(test_vals.values)
    
    # Windowing
    def make_windows(data_arr):
        n = data_arr.shape[0]
        if n <= WINDOW_SIZE:
            return np.empty((0, WINDOW_SIZE, data_arr.shape[1]))
        indexer = np.arange(WINDOW_SIZE)[None, :] + np.arange(n - WINDOW_SIZE + 1)[:, None]
        return data_arr[indexer]

    train_windows = make_windows(x_train)
    test_windows = make_windows(x_test)
    
    # Flatten for USAD
    w_size = WINDOW_SIZE * x_train.shape[1]
    z_size = WINDOW_SIZE * HIDDEN_SIZE
    
    train_windows_flat = train_windows.reshape(-1, w_size)
    test_windows_flat = test_windows.reshape(-1, w_size)
    
    # DataLoaders
    split_idx = int(0.8 * len(train_windows_flat))
    train_loader = torch.utils.data.DataLoader(
        data_utils.TensorDataset(torch.from_numpy(train_windows_flat[:split_idx]).float()),
        batch_size=BATCH_SIZE, shuffle=False
    )
    val_loader = torch.utils.data.DataLoader(
        data_utils.TensorDataset(torch.from_numpy(train_windows_flat[split_idx:]).float()),
        batch_size=BATCH_SIZE, shuffle=False
    )
    test_loader = torch.utils.data.DataLoader(
        data_utils.TensorDataset(torch.from_numpy(test_windows_flat).float()),
        batch_size=BATCH_SIZE, shuffle=False
    )
    
    # Train
    model = USAD(N_EPOCHS, w_size, z_size).to(device())
    
    print("  training model...")
    start_time = time.time()
    model.fit(train_loader, val_loader)
    training_time = time.time() - start_time
    
    print("  anomaly detection...")
    start_time = time.time()
    results_list = model.predict(test_loader)
    if len(results_list) > 0:
        scores = torch.cat(results_list).cpu().numpy()
    else:
        scores = np.array([])
    scoring_time = time.time() - start_time
    
    print(f"  Scores stats: min={scores.min():.6f}, max={scores.max():.6f}, mean={scores.mean():.6f}")
    
    # Window-based Labeling Logic
    if "label" in test_df_raw.columns:
        test_true_labels = test_df_raw["label"].to_numpy()
        label_windows = make_windows(test_true_labels.reshape(-1, 1)).reshape(-1, WINDOW_SIZE)
        y_test = (label_windows.sum(axis=1) > 0).astype(int)
    else:
        y_test = np.zeros(len(scores))

    # Align lengths
    min_len = min(len(scores), len(y_test))
    scores = scores[:min_len]
    y_test = y_test[:min_len]

    print("  finding best threshold...")
    best_threshold, best_metrics = find_best_threshold(scores, y_test)
    
    # Save Results
    results_dir_syn = "../../results/models/usad_synthetic"
    os.makedirs(results_dir_syn, exist_ok=True)
    
    full_df = pd.concat([train_df_raw, test_df_raw])
    
    # Use first feature column for visualization
    feature_cols = [c for c in train_df_raw.columns if c not in cols_to_drop]
    if feature_cols:
        complete_values = full_df[feature_cols[0]].to_numpy()
    else:
        complete_values = full_df.iloc[:, 0].to_numpy()

    if "label" in full_df.columns:
        complete_labels = full_df["label"].to_numpy()
    else:
        complete_labels = np.zeros(len(complete_values))
        
    if "timestamp" in full_df.columns:
        complete_timestamps = full_df["timestamp"].to_numpy()
    else:
        complete_timestamps = range(len(complete_values))
        
    complete_predictions = np.zeros(len(complete_values))
    complete_anomaly_scores = np.zeros(len(complete_values))
    
    # Align predictions
    pred_start_idx = len(train_df_raw) + WINDOW_SIZE - 1
    
    end_idx = pred_start_idx + len(scores)
    if end_idx > len(complete_values):
        end_idx = len(complete_values)
        scores = scores[:end_idx - pred_start_idx]
        
    complete_predictions[pred_start_idx:end_idx] = (scores > best_threshold).astype(int)
    complete_anomaly_scores[pred_start_idx:end_idx] = scores
    
    result_df = pd.DataFrame({
        'timestamp': complete_timestamps,
        'value': complete_values,
        'label': complete_labels,
        'predicted': complete_predictions,
        'anomaly_score': complete_anomaly_scores
    })
    
    output_file = os.path.join(results_dir_syn, "synthetic.csv")
    result_df.to_csv(output_file, index=False)
    print(f"  results saved to: {output_file}")
    
    print("\nResults:")
    print(f"  best_threshold: {best_threshold:.4f}")
    for k, v in best_metrics.items():
        if isinstance(v, float):
            print(f"  {k}: {v:.4f}")
        else:
            print(f"  {k}: {v}")

# Run
process_synthetic_data_usad()


Processing Synthetic Dataset (USAD)...
  Train shape: (2500, 12)
  Test shape: (2500, 12)
  training model...
Epoch [0], val_loss1: 0.0414, val_loss2: 0.0413
Epoch [1], val_loss1: 0.0417, val_loss2: -0.0000
Epoch [2], val_loss1: 0.0421, val_loss2: -0.0140
Epoch [3], val_loss1: 0.0425, val_loss2: -0.0212
Epoch [4], val_loss1: 0.0429, val_loss2: -0.0257
Epoch [5], val_loss1: 0.0434, val_loss2: -0.0288
Epoch [6], val_loss1: 0.0440, val_loss2: -0.0312
Epoch [7], val_loss1: 0.0446, val_loss2: -0.0332
Epoch [8], val_loss1: 0.0453, val_loss2: -0.0349
Epoch [9], val_loss1: 0.0461, val_loss2: -0.0365
Epoch [10], val_loss1: 0.0470, val_loss2: -0.0381
Epoch [11], val_loss1: 0.0483, val_loss2: -0.0399
Epoch [12], val_loss1: 0.0500, val_loss2: -0.0420
Epoch [13], val_loss1: 0.0521, val_loss2: -0.0444
Epoch [14], val_loss1: 0.0533, val_loss2: -0.0460
Epoch [15], val_loss1: 0.0531, val_loss2: -0.0463
Epoch [16], val_loss1: 0.0521, val_loss2: -0.0457
Epoch [17], val_loss1: 0.0508, val_loss2: -0.0449
